# Convert "Hey Nap" ONNX to TFLite

Upload your `hey_nap.onnx` file and convert it to TFLite format.

In [ ]:
# Install all dependencies
!pip install -q onnx onnx2tf tensorflow
print("✅ Dependencies installed")

In [ ]:
# Upload your ONNX file
from google.colab import files
import os

print("📤 Upload your hey_nap.onnx file...")
uploaded = files.upload()
onnx_file = list(uploaded.keys())[0]
print(f"✅ Uploaded: {onnx_file}")
print(f"   Size: {os.path.getsize(onnx_file) / 1024:.1f} KB")

In [ ]:
# Convert ONNX to TFLite
from pathlib import Path
import os

model_name = Path(onnx_file).stem
tflite_file = f"{model_name}.tflite"

print(f"🔄 Converting {onnx_file} to TFLite...")
print("\nInstalling dependencies (just to be sure)...")
!pip install -q onnx onnx2tf

print("\nConverting (this may take a minute)...")
import shlex
onnx_file_quoted = shlex.quote(onnx_file)
!onnx2tf -i {onnx_file_quoted} -o . -osd

# Check for output
output_file = f"{model_name}_float32.tflite"
if os.path.exists(output_file):
    os.rename(output_file, tflite_file)
    print(f"\n✅ Conversion successful!")
    print(f"   File: {tflite_file}")
    print(f"   Size: {os.path.getsize(tflite_file) / 1024:.1f} KB")
else:
    print("\n⚠️  onnx2tf output not found, trying alternative method...")
    !ls -la *.tflite 2>/dev/null || echo "No TFLite files found"

In [ ]:
# Alternative conversion method (if onnx2tf failed)
print("Trying alternative conversion method...")
!pip install -q onnx-tf

try:
    import onnx
    import tensorflow as tf
    from onnx_tf.backend import prepare
    import tempfile
    
    print("Loading ONNX model...")
    onnx_model = onnx.load(onnx_file)
    
    print("Converting to TensorFlow...")
    tf_rep = prepare(onnx_model)
    
    print("Exporting to SavedModel...")
    with tempfile.TemporaryDirectory() as tmp_dir:
        tf_model_path = os.path.join(tmp_dir, "saved_model")
        tf_rep.export_graph(tf_model_path)
        
        print("Converting to TFLite...")
        converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
        tflite_model = converter.convert()
        
        tflite_file = f"{model_name}.tflite"
        with open(tflite_file, 'wb') as f:
            f.write(tflite_model)
        
        print(f"\n✅ Alternative conversion successful!")
        print(f"   File: {tflite_file}")
        print(f"   Size: {os.path.getsize(tflite_file) / 1024:.1f} KB")
        
except Exception as e:
    print(f"\n❌ Alternative method failed: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Download the TFLite file
from google.colab import files
import os
import glob

tflite_file = f"{model_name}.tflite"

if os.path.exists(tflite_file):
    print(f"📥 Downloading {tflite_file}...")
    files.download(tflite_file)
    print(f"\n✅ Download complete!")
    print(f"\n🎉 Your TFLite model is ready!")
else:
    # Look for any TFLite file
    tflite_files = glob.glob("*.tflite")
    if tflite_files:
        print(f"Found: {tflite_files[0]}")
        files.download(tflite_files[0])
    else:
        print("❌ No TFLite file found")
        print("\nAvailable files:")
        !ls -la